# Evaluación de Modelos de Aprendizaje Automático

Investiga sobre las diferentes métricas que se utilizan para evaluar el desempeño de un modelo de aprendizaje automático y desarrolla una metodología que te permita extender ese análisis a cualquier tipo de solución elaborada. Aplica este procedimiento al modelo elaborado en la pregunta 1 y registra tus conclusiones en un documento electrónico de la actividad.